<a href="https://colab.research.google.com/github/MarrtinJ/fantasy-bball-opt/blob/main/bruteForceLineups_ParetoFront.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing/Importing Necessary Libraries

In [1]:
import numpy as np
import pandas as pd

In [7]:
# stop the pandas indexing/splicing warning from appearing
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('dataset2.csv')
df.shape

(43482, 27)

In [3]:
# starting with just a single date

# date = 20191022
date = 20191024
df = df[df['Date']==date]
df = df[['Date', 'Name', 'Team', 'Position', 'Salary', 'FPTS']]
df = df[df.FPTS > 0] # drop all players who have negative value
df.reset_index(inplace=True, drop=True)

df.shape

(63, 6)

# Creating DFs for each position

In [ ]:
df

,Date,Name,Team,Position,Salary,FPTS
0,20191024,Trae Young,Atl,PG,8600,62.25
1,20191024,Russell Westbrook,Hou,PG,8900,62.00
2,20191024,James Harden,Hou,PG/SG,9700,51.25
3,20191024,DAngelo Russell,GSW,PG/SG,7800,33.50
4,20191024,Stephen Curry,GSW,PG/SG,9400,33.00
...,...,...,...,...,...,...
58,20191024,Kevon Looney,GSW,PF/C,5100,15.75
59,20191024,Bruno Fernando,Atl,C,3000,14.25
60,20191024,Thon Maker,Det,PF/C,3300,12.25
61,20191024,Mfiondu Kabengele,LAC,PF/C,3000,6.75


In [4]:
Utils = df.copy()
Utils['Position'] = 'UTIL'
Utils.sort_values(by=['Salary', 'FPTS'], ascending=[True, False], inplace=True)
# Utils

In [5]:
multPositions = df[df['Position'].str.contains("/")]
multPositions.reset_index(inplace=True, drop=True)

pos1 = []
pos2 = []
for index, row in multPositions.iterrows():
  playerPos1, playerPos2 = row['Position'].split('/')
  copy1 = row.copy()
  copy1['Position'] = playerPos1
  copy2 = row.copy()
  copy2['Position'] = playerPos2
  # print(copy)
  pos1.append(copy1)
  pos2.append(copy2)
  # print(playerPos1, playerPos2)

pos1 = pd.DataFrame(pos1)
pos2 = pd.DataFrame(pos2)
print(multPositions.shape, pos1.shape, pos2.shape)


(48, 6) (48, 6) (48, 6)


In [ ]:
# pos1

In [ ]:
# pos2

# New Section

In [8]:
# for the players that only play one position, filter out players that had less value than their peers
onePosition = df[ df['Position'].str.contains('/')==False ]
onePosition.sort_values(by=['Salary', 'FPTS'], ascending=[True, False], inplace=True)
onePositionPG = onePosition[ onePosition['Position'] == 'PG' ]
onePositionSG = onePosition[ onePosition['Position'] == 'SG' ]
onePositionSF = onePosition[ onePosition['Position'] == 'SF' ]
onePositionPF = onePosition[ onePosition['Position'] == 'PF' ]
onePositionC = onePosition[ onePosition['Position'] == 'C' ]
onePositionG = pd.concat([onePositionPG, onePositionSG], ignore_index=True)
onePositionG.sort_values(by=['Salary', 'FPTS'], ascending=[True, False], inplace=True)
onePositionF = pd.concat([onePositionSF, onePositionPF], ignore_index=True)
onePositionF.sort_values(by=['Salary', 'FPTS'], ascending=[True, False], inplace=True)
onePositionUtil = onePosition.copy()

print(onePositionPG.shape)
print(onePositionSG.shape)
print(onePositionSF.shape)
print(onePositionPF.shape)
print(onePositionC.shape)
print(onePositionG.shape)
print(onePositionF.shape)
print(onePositionUtil.shape)

(5, 6)
(1, 6)
(1, 6)
(0, 6)
(8, 6)
(6, 6)
(1, 6)
(15, 6)


In [9]:
onePositionPG.drop_duplicates(subset='Salary', keep='first', inplace=True)
onePositionSG.drop_duplicates(subset='Salary', keep='first', inplace=True)
onePositionSF.drop_duplicates(subset='Salary', keep='first', inplace=True)
onePositionPF.drop_duplicates(subset='Salary', keep='first', inplace=True)
onePositionC.drop_duplicates(subset='Salary', keep='first', inplace=True)
onePositionG.drop_duplicates(subset='Salary', keep='first', inplace=True)
onePositionF.drop_duplicates(subset='Salary', keep='first', inplace=True)
onePositionUtil.drop_duplicates(subset='Salary', keep='first', inplace=True)


print(onePositionPG.shape)
print(onePositionSG.shape)
print(onePositionSF.shape)
print(onePositionPF.shape)
print(onePositionC.shape)
print(onePositionG.shape)
print(onePositionF.shape)
print(onePositionUtil.shape)

(4, 6)
(1, 6)
(1, 6)
(0, 6)
(8, 6)
(4, 6)
(1, 6)
(11, 6)


In [10]:
onePosList = [onePositionPG, onePositionSG, onePositionSF, onePositionPF, onePositionC, onePositionG, onePositionF, onePositionUtil]

In [11]:
currentPlayers = pd.concat([onePosition, pos1, pos2], ignore_index=True)
currentPlayers.sort_values(['Name', 'Team', 'Position'], na_position='first', inplace=True, ignore_index=True)
currentPlayers.shape

(111, 6)

In [12]:
Gs = currentPlayers[currentPlayers['Position'].isin(['PG', 'SG'])]
Gs.drop_duplicates(subset='Name', inplace=True)
Gs.reset_index(inplace=True, drop=True)
Gs['Position'] = 'G'
# Gs

In [13]:
Fs = currentPlayers[currentPlayers['Position'].isin(['SF', 'PF'])]
Fs.drop_duplicates(subset='Name', inplace=True)
Fs.reset_index(inplace=True, drop=True)
Fs['Position'] = 'F'
# Fs

In [14]:
currentPlayers = pd.concat([currentPlayers, Gs, Fs, Utils], ignore_index=True)
currentPlayers.sort_values(['Name', 'Team', 'Position'], na_position='first', inplace=True, ignore_index=True)

In [15]:
currentPlayers

,Date,Name,Team,Position,Salary,FPTS
0,20191024,Alex Len,Atl,C,5500,18.50
1,20191024,Alex Len,Atl,UTIL,5500,18.50
2,20191024,Andre Drummond,Det,C,8300,48.50
3,20191024,Andre Drummond,Det,UTIL,8300,48.50
4,20191024,Austin Rivers,Hou,G,3200,5.50
...,...,...,...,...,...,...
242,20191024,Tyson Chandler,Hou,UTIL,3100,16.25
243,20191024,Vince Carter,Atl,F,3100,1.00
244,20191024,Vince Carter,Atl,PF,3100,1.00
245,20191024,Vince Carter,Atl,SF,3100,1.00


# DFS Setup / Helper Functions

In [16]:
# incr determines if the players are each position are sorted by increasing salary
incr = [True, False]

PGs = currentPlayers[currentPlayers['Position'] == 'PG']
PGs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
PGs.reset_index(inplace=True, drop=True)

SGs = currentPlayers[currentPlayers['Position'] == 'SG']
SGs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
SGs.reset_index(inplace=True, drop=True)

SFs = currentPlayers[currentPlayers['Position'] == 'SF']
SFs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
SFs.reset_index(inplace=True, drop=True)

PFs = currentPlayers[currentPlayers['Position'] == 'PF']
PFs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
PFs.reset_index(inplace=True, drop=True)

Cs = currentPlayers[currentPlayers['Position'] == 'C']
Cs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
Cs.reset_index(inplace=True, drop=True)

Gs = currentPlayers[currentPlayers['Position'] == 'G']
Gs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
Gs.reset_index(inplace=True, drop=True)

Fs = currentPlayers[currentPlayers['Position'] == 'F']
Fs.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
Fs.reset_index(inplace=True, drop=True)

Utils = currentPlayers[currentPlayers['Position'] == 'UTIL']
Utils.sort_values(by=['Salary', 'FPTS'], ascending=incr, inplace=True)
Utils.reset_index(inplace=True, drop=True)


In [ ]:
# Utils

In [ ]:
# onePositionUtil

In [ ]:
# Utils[['Name', 'Salary', 'FPTS']].to_csv(f'test.csv', line_terminator='\n', index=False)

## Pareto Front

Considering all players in a given position from left to right (after sorting them by ascending salary), we ignore any player whose score is not better than the best player to its left so far (with the nuance that a player who plays in multiple positions should never be used to determine the best score so far). You can then make a new list consisting only of the players who were not eliminated by this process (or add the players who were not eliminated to a new list as you go).

In [17]:
# takes as input a DF containing  all the players that play position i
#   and a list of player names who only play position i
# returns a list of player names that are not pareto-dominated
def paretoFilter(posDF, singlePosNames):
  discard = []
  # print(singlePosNames[0])
  if not singlePosNames: # no players listed under just that one position
    return discard
  else:
    curBest = posDF[posDF['Name'] == singlePosNames[0]].iloc[0] # the first best player
    # print(curBest)
    for index, row in posDF.iterrows():
      # if the current player produces more fantasy points and only plays that specific position
      if row['FPTS'] > curBest['FPTS'] and row['Name'] in singlePosNames:
        curBest = row
        # print('Updating current best')
        # print(curBest)
      else:
        # if the current player produces less fantasy points than the best player
        if row['FPTS'] <= curBest['FPTS'] and row['Name'] != curBest['Name']:
            # print('Dropping {}'.format(row['Name']))
            # print('Cur: {}, {} Best:{} {}'.format(row['Salary'], row['FPTS'], curBest['Salary'], curBest['FPTS']))
            discard.append(row['Name'])
    return discard

In [19]:
l1 = [PGs, SGs, SFs, PFs, Cs, Gs, Fs, Utils]

for y in range (8):
  onePosNames = onePosList[y].Name.to_list()
  # print(onePosNames)
  print(l1[y].iloc[0].Position)
  print('before: {}'.format(l1[y].shape))
  test = paretoFilter(l1[y], onePosNames)
  l1[y] = l1[y][~l1[y]['Name'].isin(test)]
  # print(len(test))
  print('after: {}'.format(l1[y].shape))  

PG
before: (15, 6)
after: (10, 6)
SG
before: (27, 6)
after: (22, 6)
SF
before: (29, 6)
after: (23, 6)
PF
before: (21, 6)
after: (21, 6)
C
before: (19, 6)
after: (9, 6)
G
before: (33, 6)
after: (22, 6)
F
before: (40, 6)
after: (33, 6)
UTIL
before: (63, 6)
after: (19, 6)


In [20]:
# minPrices holds the minimum budget required to keep searching at each position
#   where the number at index i represents the sum of the least expensive players at position index >= i

# maxPrices holds the minimum budget required to afford the highest scoring players in the remaining positions
#   where the number at index i represents the sum of the most expensive players at position index >= i

minSalaries = []
maxSalaries = []
l2 = []
position_order = []

l3 = []

for i in range(8):
  # could also consider sorting by the mean salary of each position
  pos_max = l1[i].Salary.max()
  l2.append((pos_max, l1[i]))

# sort the positions in descending order by max salary
l2 = sorted(l2, key=lambda x: x[0], reverse=True)

cheapest_sum = 0
expensive_sum = 0
for i in range(len(l2)):
  # keeping track of the sum of salaries for the most expensive players in each position
  pos_max = l2[i][0]
  expensive_sum += pos_max
  maxSalaries.insert(0, expensive_sum)

  # removing the max sal info, retaining just the dataframe
  l2[i] = l2[i][1]

  l3.append(l2[i].sort_values(by=['FPTS'], ascending=False))

  # keeping track of the sum of salaries for the least expensive players in each position
  pos_min = l2[i].Salary.min()
  cheapest_sum += pos_min
  minSalaries.insert(0, cheapest_sum)

  # keeping track of the sorted order of the positions
  pos = l2[i]['Position'].iloc[0]
  position_order.append(pos)

  print(pos, pos_min, pos_max)

# l2
# position_order
# maxSalaries
# minSalaries
# l3

SF 3000 10300
PF 3000 10300
F 3000 10300
UTIL 3000 10300
SG 3000 9700
PG 3000 8600
G 3000 8600
C 3000 8300


In [21]:
# players[i][j] represents the ith player in the jth position
# players[i] sorted in ascending order by salary

players = []
players_per_pos = []

for i in range(len(l2)):
  players_list = l2[i]['Name'].to_list()
  print(position_order[i], len(players_list))
  players.append(players_list)
  players_per_pos.append(len(players_list))

players_per_pos = np.array(players_per_pos)

SF 23
PF 21
F 33
UTIL 19
SG 22
PG 10
G 22
C 9


In [22]:
# checkDuplicates takes in a list of players, checks if list contains duplicates
# if yes return True, else return False
def checkDuplicates(players):
  setOfPlayers = set()
  for player in players:
      if player in setOfPlayers:
          return True
      else:
          setOfPlayers.add(player)         
  return False

In [23]:
# checkSalary takes in a list of players, checks if salary > 50000
# if yes return true, else return false
def checkSalary(players):
  total_salary = getSalary(players)
  if total_salary > 50000:
    return True
  return False

# getSalary takes in a list of players, returns the combined salary of all players
def getSalary(players):
  total_salary = 0
  for player in players:
    player_sal = currentPlayers[currentPlayers['Name'] == player].iloc[0].Salary
    total_salary += player_sal
  return total_salary

In [24]:
def getSingleScore(player):
  return currentPlayers[currentPlayers['Name'] == player].iloc[0].FPTS

def getSingleSalary(player):
  return currentPlayers[currentPlayers['Name'] == player].iloc[0].Salary

In [25]:
# getScore takes in a dict of players, returns the fantasy points scored by players
def getScore(players):
  total_score = 0
  for player in players.values():
    player_score = getSingleScore(player)
    # print(player, player_score)
    total_score += player_score
  return total_score

# Inverted DFS

In [26]:
players_per_pos

array([23, 21, 33, 19, 22, 10, 22,  9])

In [27]:
global numLineups
numLineups = 0

In [30]:
MAX_POSITION_INDEX = 7

global numLineups
numLineups = 0

# invertedDFS(players, 0, 0, 50000)
def invertedDFS(players, pos_index, player_index, budget):

  # print(pos_index, len(players[pos_index]))
  player = players[pos_index][player_index]

  salary = getSingleSalary(player)
  newBudget = budget - salary

  # optimization 1: instead of comparing with 0, compare with sum(min_budget of each remaining position)
  #   can be computed ahead of time for each level
  if pos_index < MAX_POSITION_INDEX and newBudget < minSalaries[pos_index+1]:
    # print('out of budget at position {}'.format(pos_index))
    return dict(), np.NINF

  # optimization 3: two separate cases: if I have enough budget to pick the max salary from every remaining level, no need for recursion, just pick best score from every lower level
  # otherwise, do the recursive search in the next block
  if pos_index < MAX_POSITION_INDEX and newBudget > maxSalaries[pos_index+1]:
    # print('can afford best players at position {}'.format(pos_index))

    assignment = dict()
    assignment[pos_index] = player
    for i in range(pos_index+1, 8):
      for index, row in l3[i].iterrows():
        if row['Name'] not in assignment.values():
          assignment[i] = row['Name']
          break
    # print(assignment, getScore(assignment))
    numLineups += 1
    if numLineups%10 == 0:
      print(numLineups)
    return assignment, getScore(assignment)

  if pos_index == MAX_POSITION_INDEX:
    assignment = {pos_index: player}
    # print(pos_index, player_index, player)
    numLineups += 1
    if numLineups%10 == 0:
      print(numLineups)
    return assignment, getScore(assignment)
  else:
    max_score = 0
    best_lineup = dict()
    for j in range(len(players[pos_index + 1])):
    # for j in range(8):
      a, s = invertedDFS(players, pos_index+1, j, newBudget) 
      # optimization 2: if a returns None (a is empty), break out of the current loop
      if not a:
        # print('next level returned none')
        numLineups += 1
        if numLineups%10 == 0:
          print(numLineups)
        break
      if s > max_score and a and player not in a.values(): #duplicate check
        max_score = s
        best_lineup = a
        # print(max_score, best_lineup)
    if best_lineup:
      best_lineup[pos_index]=player
      numLineups += 1
      if numLineups%10 == 0:
        print(numLineups)
      return best_lineup, max_score + getSingleScore(player)
    else:
      numLineups += 1
      if numLineups%10 == 0:
        print(numLineups)
      return dict(), np.NINF

In [31]:
invertedDFS(players, 0, 0, 50000)

UnboundLocalError: local variable 'numLineups' referenced before assignment

# Test

In [ ]:
res = {'Anthony Davis', 'Danny Green', 'Josh Hart', 'Kawhi Leonard', 'LeBron James', 'Maurice Harkless', 'Nickeil Alexander-Walker', 'Nicolo Melli'}
test = df[df['Name'].isin(res)]
test
# test['Salary'].sum()
# getScore(res)

,Name,Team,Position,Salary,FPTS
0,LeBron James,LAL,PG/SF,9900,43.25
4,Nickeil Alexander-Walker,Nor,PG/SG,3400,15.00
9,Kawhi Leonard,LAC,SF,9600,48.50
10,Danny Green,LAL,SG/SF,5300,46.25
12,Josh Hart,Nor,SG/SF,3200,33.50
13,Maurice Harkless,LAC,SF/PF,3700,28.00
21,Anthony Davis,LAL,PF/C,10200,51.00
23,Nicolo Melli,Nor,PF/C,3000,24.25
